In [2]:
# Importing Pandas for loading datasets
import pandas as pd 

In [13]:
# Loading dataset
df = pd.read_csv("AB_NYC_2019.csv")

In [14]:
# 1. Data Integrity: Ensuring the accuracy, consistency, and reliability of data throughout the cleaning process.

In [15]:
# 1. Ensure correct data types

df['price'] = pd.to_numeric(df['price'], errors='coerce')
df['minimum_nights'] = pd.to_numeric(df['minimum_nights'], errors='coerce')
df['number_of_reviews'] = pd.to_numeric(df['number_of_reviews'], errors='coerce')

# 2. Handleing invalid or inconsistent values
# (Price must be positive)
df = df[df['price'] > 0]

# Minimum nights must be at least 1
df = df[df['minimum_nights'] >= 1]

# 3. Check value ranges and constraints
# Latitude and longitude must be within valid ranges
df = df[(df['latitude'].between(-90, 90)) & 
        (df['longitude'].between(-180, 180))]

# 4. Removeing corrupted or incomplete records
# Drop rows missing critical identifiers
df = df.dropna(subset=['id', 'host_id'])

# 5. Final integrity check

print("Data types after cleaning:\n", df.dtypes)
print("\nRemaining missing values:\n", df.isnull().sum())
print("\nClean dataset shape:", df.shape)

Data types after cleaning:
 id                                  int64
name                               object
host_id                             int64
host_name                          object
neighbourhood_group                object
neighbourhood                      object
latitude                          float64
longitude                         float64
room_type                          object
price                               int64
minimum_nights                      int64
number_of_reviews                   int64
last_review                        object
reviews_per_month                 float64
calculated_host_listings_count      int64
availability_365                    int64
dtype: object

Remaining missing values:
 id                                    0
name                                 16
host_id                               0
host_name                            21
neighbourhood_group                   0
neighbourhood                         0
latitude          

In [16]:
# 2. Missing Data Handling: Dealing with missing values by either imputing them or making 
#informed decisions on how to handle gaps in the dataset.

In [12]:
# 1. Identifying missing values
print("Missing values per column:")
print(df.isnull().sum())

# 2. Impute missing values
# Useing median for numerical column (robust to outliers)
df['reviews_per_month'] = df['reviews_per_month'].fillna(
    df['reviews_per_month'].median()
)

# Useing mode for categorical column
df['neighbourhood_group'] = df['neighbourhood_group'].fillna(
    df['neighbourhood_group'].mode()[0]
)

# 3. Informed removal of records
# Drop rows where critical information is missing
df = df.dropna(subset=['latitude', 'longitude'])

# 4. Final check
print("\nMissing values after handling:")
print(df.isnull().sum())
print("\nClean dataset shape:", df.shape)

Missing values per column:
id                                    0
name                                 16
host_id                               0
host_name                            21
neighbourhood_group                   0
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
last_review                       10051
reviews_per_month                 10051
calculated_host_listings_count        0
availability_365                      0
dtype: int64

Missing values after handling:
id                                    0
name                                 16
host_id                               0
host_name                            21
neighbourhood_group                   0
neighbourhood                         0
latitude                              0
longitud

In [17]:
# 3. Duplicate Removal: Identifying and eliminating duplicate records to maintain data uniqueness.

In [19]:
# Check total duplicate rows
duplicate_count = df.duplicated().sum()
print("Total duplicate rows:", duplicate_count)

# Identifying duplicates based on key columns
key_duplicates = df.duplicated(subset=['id'], keep=False)
print("Duplicate IDs found:", key_duplicates.sum())

# Removeing duplicate records
df = df.drop_duplicates()

# Removeing duplicates based on unique identifier (keep first occurrence)
df = df.drop_duplicates(subset=['id'], keep='first')

# Final validation
print("\nDataset shape after duplicate removal:", df.shape)
print("Remaining duplicate IDs:", df['id'].duplicated().sum())

Total duplicate rows: 0
Duplicate IDs found: 0

Dataset shape after duplicate removal: (48884, 16)
Remaining duplicate IDs: 0


In [20]:
# 4. Standardization: Consistent formatting and units across the dataset for accurate analysis.

In [22]:
# 1. Standardizeing text formatting
df['neighbourhood_group'] = (
    df['neighbourhood_group']
    .str.strip()        # remove extra spaces
    .str.title()        # consistent capitalization
)

df['room_type'] = (
    df['room_type']
    .str.strip()
    .str.lower()        # consistent lowercase
)

# 2. Standardizeing date formats
df['last_review'] = pd.to_datetime(
    df['last_review'], errors='coerce'
)

# 3. Standardizeing numerical units
# Ensure price is numeric and rounded to two decimals
df['price'] = pd.to_numeric(df['price'], errors='coerce').round(2)

# Ensure availability is integer (days)
df['availability_365'] = df['availability_365'].astype(int)

# 4. Final validation
print("Standardized data types:\n")
print(df.dtypes)

print("\nSample standardized data:")
print(df[['neighbourhood_group', 'room_type', 'price', 'last_review']].head())

Standardized data types:

id                                         int64
name                                      object
host_id                                    int64
host_name                                 object
neighbourhood_group                       object
neighbourhood                             object
latitude                                 float64
longitude                                float64
room_type                                 object
price                                      int64
minimum_nights                             int64
number_of_reviews                          int64
last_review                       datetime64[ns]
reviews_per_month                        float64
calculated_host_listings_count             int64
availability_365                           int64
dtype: object

Sample standardized data:
  neighbourhood_group        room_type  price last_review
0            Brooklyn     private room    149  2018-10-19
1           Manhattan  entire hom

In [23]:
# 5. Outlier Detection: Identifying and addressing outliers that may skew analysis or model performance.

In [24]:
# 1. Detecting outliers using IQR
Q1 = df['price'].quantile(0.25)
Q3 = df['price'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Identifying outliers
outliers = df[(df['price'] < lower_bound) | (df['price'] > upper_bound)]
print("Number of price outliers:", outliers.shape[0])

# 2. Address outliers
# Option 1: Remove outliers
df_no_outliers = df[
    (df['price'] >= lower_bound) & (df['price'] <= upper_bound)
]

# Option 2: Cap outliers (winsorization)
df['price_capped'] = df['price'].clip(
    lower=lower_bound,
    upper=upper_bound
)

# 3. Final check
print("Dataset shape after removing outliers:", df_no_outliers.shape)

Number of price outliers: 2972
Dataset shape after removing outliers: (45912, 16)


In [5]:
import json

# Load JSON file
with open("CA_category_id.json", "r") as file:
    data = json.load(file)

# Convert to DataFrame
df = pd.json_normalize(data['items'])

print(df.head())

                    kind                                               etag  \
0  youtube#videoCategory  "ld9biNPKjAjgjV7EZ4EKeEGrhao/Xy1mB4_yLrHy_BmKm...   
1  youtube#videoCategory  "ld9biNPKjAjgjV7EZ4EKeEGrhao/UZ1oLIIz2dxIhO45Z...   
2  youtube#videoCategory  "ld9biNPKjAjgjV7EZ4EKeEGrhao/nqRIq97-xe5XRZTxb...   
3  youtube#videoCategory  "ld9biNPKjAjgjV7EZ4EKeEGrhao/HwXKamM1Q20q9BN-o...   
4  youtube#videoCategory  "ld9biNPKjAjgjV7EZ4EKeEGrhao/9GQMSRjrZdHeb1OEM...   

   id         snippet.channelId     snippet.title  snippet.assignable  
0   1  UCBR8-60-B28hp2BmDPdntcQ  Film & Animation                True  
1   2  UCBR8-60-B28hp2BmDPdntcQ  Autos & Vehicles                True  
2  10  UCBR8-60-B28hp2BmDPdntcQ             Music                True  
3  15  UCBR8-60-B28hp2BmDPdntcQ    Pets & Animals                True  
4  17  UCBR8-60-B28hp2BmDPdntcQ            Sports                True  


In [26]:
# Ensure correct data types
df['id'] = df['id'].astype(int)
df['snippet.assignable'] = df['snippet.assignable'].astype(bool)

# Remove records with missing critical fields
df = df.dropna(subset=['id', 'snippet.title'])

print(df.dtypes)


kind                  object
etag                  object
id                     int64
snippet.channelId     object
snippet.title         object
snippet.assignable      bool
dtype: object


In [27]:
# Check missing values
print(df.isnull().sum())

# Fill missing channel IDs with 'Unknown'
df['snippet.channelId'] = df['snippet.channelId'].fillna("Unknown")


kind                  0
etag                  0
id                    0
snippet.channelId     0
snippet.title         0
snippet.assignable    0
dtype: int64


In [28]:
# Check duplicates based on category ID
print("Duplicate IDs:", df['id'].duplicated().sum())

# Remove duplicates
df = df.drop_duplicates(subset=['id'], keep='first')


Duplicate IDs: 0


In [29]:
# Standardize text formatting
df['snippet.title'] = (
    df['snippet.title']
    .str.strip()
    .str.title()
)

# Rename columns for clarity
df = df.rename(columns={
    'snippet.title': 'category_name',
    'snippet.assignable': 'assignable'
})

In [30]:
# Check invalid IDs (should be positive)
invalid_ids = df[df['id'] <= 0]
print("Invalid IDs found:", invalid_ids.shape[0])

# Check unexpected boolean values
print(df['assignable'].value_counts())


Invalid IDs found: 0
assignable
False    17
True     14
Name: count, dtype: int64


In [31]:
print(df.head())
print("Final dataset shape:", df.shape)

                    kind                                               etag  \
0  youtube#videoCategory  "ld9biNPKjAjgjV7EZ4EKeEGrhao/Xy1mB4_yLrHy_BmKm...   
1  youtube#videoCategory  "ld9biNPKjAjgjV7EZ4EKeEGrhao/UZ1oLIIz2dxIhO45Z...   
2  youtube#videoCategory  "ld9biNPKjAjgjV7EZ4EKeEGrhao/nqRIq97-xe5XRZTxb...   
3  youtube#videoCategory  "ld9biNPKjAjgjV7EZ4EKeEGrhao/HwXKamM1Q20q9BN-o...   
4  youtube#videoCategory  "ld9biNPKjAjgjV7EZ4EKeEGrhao/9GQMSRjrZdHeb1OEM...   

   id         snippet.channelId     category_name  assignable  
0   1  UCBR8-60-B28hp2BmDPdntcQ  Film & Animation        True  
1   2  UCBR8-60-B28hp2BmDPdntcQ  Autos & Vehicles        True  
2  10  UCBR8-60-B28hp2BmDPdntcQ             Music        True  
3  15  UCBR8-60-B28hp2BmDPdntcQ    Pets & Animals        True  
4  17  UCBR8-60-B28hp2BmDPdntcQ            Sports        True  
Final dataset shape: (31, 6)
